In [1]:
from pyspark.sql import SparkSession
#.master("spark://spark-master:7077") \

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Spark with PostgreSQL") \
    .master("local[*]") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.7.4.jar") \
    .getOrCreate()

# JDBC connection properties
jdbc_url = "jdbc:postgresql://psql-postgres-1:5432/mydatabase"
connection_properties = {
    "user": "myuser",
    "password": "mysecretpassword",
    "driver": "org.postgresql.Driver"
}

# Load a table into Spark
df = spark.read.jdbc(url=jdbc_url, table="transactions", properties=connection_properties)
df.show(1)


+--------------+-------+------+--------+-------------------+
|transaction_id|user_id|amount|    type|          timestamp|
+--------------+-------+------+--------+-------------------+
|    1704169733|      9|361.83|transfer|2024-01-01 20:28:53|
+--------------+-------+------+--------+-------------------+
only showing top 1 row



In [2]:
spark.version

'3.5.0'

In [9]:
import os

# Specify the path you want to check
path = "/opt/spark/jars/postgresql-42.7.4.jar"

# Check if the path exists
if os.path.exists(path):
    print("Path exists!")
else:
    print("Path does not exist.")


Path exists!


In [15]:
import psycopg2
conn = psycopg2.connect(
    host="psql-postgres-1",
    port=5432,
    user="myuser",
    password="mysecretpassword",
    dbname="mydatabase"
)
print("Connection successful!")

cur = conn.cursor()

cur.execute("""
    SELECT * FROM transactions;
""")

rows = cur.fetchall()

# Print each row
for i, row in enumerate(rows):
    if i == 10:
        break
    print(row)
    

# Commit the changes and close the connection
conn.commit()
conn.close()


Connection successful!
(1704169733, 9, Decimal('361.83'), 'transfer', datetime.datetime(2024, 1, 1, 20, 28, 53))
(1706848193, 77, Decimal('55.28'), 'transfer', datetime.datetime(2024, 2, 1, 20, 29, 53))
(1709353853, 69, Decimal('82.96'), 'purchase', datetime.datetime(2024, 3, 1, 20, 30, 53))
(1712028713, 88, Decimal('36.10'), 'transfer', datetime.datetime(2024, 4, 1, 20, 31, 53))
(1714620773, 22, Decimal('94.44'), 'transfer', datetime.datetime(2024, 5, 1, 20, 32, 53))
(1717299233, 73, Decimal('408.42'), 'purchase', datetime.datetime(2024, 6, 1, 20, 33, 53))
(1719891293, 64, Decimal('954.82'), 'transfer', datetime.datetime(2024, 7, 1, 20, 34, 53))
(1722569753, 59, Decimal('410.94'), 'purchase', datetime.datetime(2024, 8, 1, 20, 35, 53))
(1725248213, 16, Decimal('797.11'), 'purchase', datetime.datetime(2024, 9, 1, 20, 36, 53))
(1727840273, 42, Decimal('802.94'), 'purchase', datetime.datetime(2024, 10, 1, 20, 37, 53))


In [8]:
from pyspark.sql import SparkSession

spark.catalog.clearCache()

if 'spark' in globals():
    spark.stop()
    
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Spark Test") \
    .master("local[*]") \
    .getOrCreate()

# Create a simple DataFrame
data = [("Alice", 29), ("Bob", 35), ("Cathy", 45)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

# Perform a simple transformation: filter by age > 30
df_filtered = df.filter(df["Age"] > 30)

# Show the filtered DataFrame
df_filtered.show()

# Stop the Spark session when done
spark.stop()


+-----+---+
| Name|Age|
+-----+---+
|Alice| 29|
|  Bob| 35|
|Cathy| 45|
+-----+---+

+-----+---+
| Name|Age|
+-----+---+
|  Bob| 35|
|Cathy| 45|
+-----+---+



In [5]:
if 'spark' in globals():
    spark.stop()

NameError: name 'pyspark' is not defined

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("SimpleApp").getOrCreate()

data = [("Alice", 1), ("Bob", 2), ("Charlie", 3)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)
df.show()


+-------+---+
|   Name|Age|
+-------+---+
|  Alice|  1|
|    Bob|  2|
|Charlie|  3|
+-------+---+



In [ ]:
# pip install psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.7 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.
